In [3]:
import scipy.stats as st
import numpy as np
import pandas as pd
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS


In [121]:
#import lstop data
lstops = pd.read_csv('../data/lstops.csv', delimiter=',', quotechar='"')
#check column
lstops["Location"] = lstops["Location"].str.replace("(","") 
lstops["Location"] = lstops["Location"].str.replace(")","") 
lat_list = []
lon_list = []
for each in lstops["Location"]:
    x = each.split(",")
    lat = float(x[0])
    lon = float(x[1])
    lat_list.append(lat)
    lon_list.append(lon)

lstops["Lat"] = lat_list
lstops["Lon"] = lon_list

In [122]:
#import ca to zip data
ca_to_zip = pd.read_csv('../data/ca_zip.csv',header=None)
ca_to_zip = ca_to_zip.drop(ca_to_zip.index[0])
ca_to_zip.columns = ["Community Area", "Zipcode", "Pop2010"]


In [124]:
#adapted from:
#https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python?
#fbclid=IwAR25An6-uDiY1Lpy5EVPUG15RcrRKj5fL1M86Bt3Svqy_iVRi1YLLfviRT0
df = lstops

gis = GIS()

def get_zip(df, lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]), "y":float(df[lat_field]), "spatialReference":{"wkid": 4326}})))
    return location['address']['Postal']

In [125]:
#apply zipcodes
lstops["zips"] = df.apply(get_zip, axis=1, lat_field='Lat', lon_field='Lon')
#merge based on ca_to_zip
lstops = lstops.merge(ca_to_zip, left_on="zips", right_on="Zipcode")
#output csv
lstops.to_csv("../data/lstops_localized.csv",index=False)